In [1]:
#| default_exp data_loader

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
from glob import glob
import os
from typing import List, Union
import warnings

import numpy as np
import pandas as pd

from pheno_utils.config import *

In [4]:
#| export

class DataLoader:
    """
    Class to load multiple tables from a dataset and allows to easily access
    their fields.

    Args:
    
        dataset (str): The name of the dataset to load.
        cohort (str, optional): The name of the cohort within the dataset. Defaults to '10k'.
        base_path (str, optional): The base path where the data is stored. Defaults to '/home/ec2-user/studies'.
        age_sex_dataset (str, optional): The name of the dataset to use for computing age and sex. Defaults to 'Population_Characteristics'.
        unique_index (bool, optional): Whether to ensure the index of the data is unique. Defaults to False.
        valid_dates (bool, optional): Whether to ensure that all timestamps in the data are valid dates. Defaults to False.
        valid_stage (bool, optional): Whether to ensure that all research stages in the data are valid. Defaults to False.
        errors (str, optional): Whether to raise an error or issue a warning if missing data is encountered.
            Possible values are 'raise' and 'warn'. Defaults to 'raise'.

    Attributes:
    
        dict (pd.DataFrame): The data dictionary for the dataset, containing information about each field.
        dfs (dict): A dictionary of dataframes, one for each table in the dataset.
        fields (list): A list of all fields in the dataset.
        dataset (str): The name of the dataset being used.
        cohort (str): The name of the cohort being used.
        base_path (str): The base path where the data is stored.
        age_sex_dataset (str): The name of the dataset being used to compute age and sex.
        unique_index (bool): Whether to ensure the index of the data is unique.
        valid_dates (bool): Whether to ensure that all timestamps in the data are valid dates.
        valid_stage (bool): Whether to ensure that all research stages in the data are valid.
        errors (str): Whether to raise an error or issue a warning if missing data is encountered.
    """

    def __init__(
        self,
        dataset: str,
        cohort: str = '10k',
        base_path: str = DATASETS_PATH,
        age_sex_dataset: str = POPULATION_DATASET,
        skip_dfs: List[str] = [],
        unique_index: bool = False,
        valid_dates: bool = False,
        valid_stage: bool = False,
        errors: str = 'raise'
    ) -> None:
        self.dataset = dataset
        self.cohort = cohort
        self.base_path = base_path
        self.age_sex_dataset = age_sex_dataset
        self.skip_dfs = skip_dfs
        self.unique_index = unique_index
        self.valid_dates = valid_dates
        self.valid_stage = valid_stage
        self.errors = errors

        self.__load_dictionary__()
        self.__load_dataframes__()
        if self.age_sex_dataset is not None:
            self.__load_age_sex__()

    def load_sample_data(
        self,
        field_name: str,
        participant_id: Union[str, List[str]],
        research_stage: Union[None, str, List[str]] = None,
        array_index: Union[None, int, List[int]] = None,
        load_func: callable = pd.read_parquet,
        concat: bool = True
    ) -> Union[pd.DataFrame, None]:
        """
        Load time series or bulk data for sample(s).

        Args:
            field_name (str): The name of the field to load.
            participant_id (str or list): The participant ID or IDs to load data for.
            research_stage (str or list, optional): The research stage or stages to load data for.
            array_index (int or list, optional): The array index or indices to load data for.
            load_func (callable, optional): The function to use to load the data. Defaults to pd.read
            concat (bool, optional): Whether to concatenate the data into a single DataFrame. Defaults to True.
        """
        query_str = 'participant_id in @participant_id'
        if not isinstance(participant_id, list):
            participant_id = [participant_id]
        if research_stage is not None:
            if not isinstance(research_stage, list):
                research_stage = [research_stage]
            query_str += ' and research_stage in @research_stage'
        if array_index is not None:
            if not isinstance(array_index, list):
                array_index = [array_index]
            query_str += ' and array_index in @array_index'

        sample = self[[field_name] + ['participant_id']].query(query_str)
        missing_participants = np.setdiff1d(participant_id, sample['participant_id'].unique())
        sample = os.path.join(
            self.base_path,
            self.dataset,
            self.cohort) + '/' + sample.iloc[:, 0]

        if len(missing_participants):
            if self.errors == 'raise':
                raise ValueError(f'Missing samples: {missing_participants}')
            elif self.errors == 'warn':
                warnings.warn(f'Missing samples: {missing_participants}')
            if len(sample) == 0:
                return None

        data = []
        for p in sample.unique():
            try:
                data.append(load_func(p))
            except Exception as e:
                if self.errors == 'raise':
                    raise e
                elif self.errors == 'warn':
                    warnings.warn(f'Error loading {p}: {e}')
        if concat:
            data = pd.concat(data, axis=0)
        return data

    def __repr__(self):
        """
        Return string representation of object

        Returns:
            str: String representation of object
        """
        return self.__str__()

    def __str__(self):
        """
        Return string representation of object

        Returns:
            str: String representation of object
        """
        return f'DataLoader for {self.dataset} with' +\
            f'\n{len(self.fields)} fields\n{len(self.dfs)} tables: {list(self.dfs.keys())}'

    def __getitem__(self, fields):
        """
        Return data for the specified fields from all tables

        Args:
            fields (Union[str, List[str]]): Fields to return

        Returns:
            pd.DataFrame: Data for the specified fields from all tables
        """
        if isinstance(fields, str):
            fields = [fields]
        return self.__get_data__(fields)

    def __get_data__(self, fields):
        """
        Return data for the specified fields from all tables

        Args:
            fields (List[str]): Fields to return

        Returns:
            pd.DataFrame: Data for the specified fields from all tables
        """
        data = []
        for df in self.dfs.values():
            data.append(df[df.columns.intersection(fields)])
        data = pd.concat(data, axis=1)
        data = data.loc[:, ~data.columns.duplicated()]

        fields_in_index = np.intersect1d(fields, data.index.names)
        for field in fields_in_index:
            data[field] = data.index.get_level_values(field)

        not_found = np.setdiff1d(fields, data.columns)
        if len(not_found):
            if self.errors == 'raise':
                raise KeyError(f'Fields not found: {not_found}')
            elif self.errors == 'warn':
                warnings.warn(f'Fileds not found: {not_found}')

        return data

    def __load_age_sex__(self) -> None:
        """
        Add sex and compute age from birth date.
        """
        age_path = os.path.join(
            self.base_path,
            self.age_sex_dataset,
            self.cohort,
            'events.parquet')
        age_df = pd.read_parquet(age_path)
        align_df = self.dfs[list(self.dfs)[0]]

        # TODO: check if research stage is "continuous"
        if ('research_stage' in align_df.columns) or ('research_stage' in align_df.index.names):
            self.dfs['age_sex'] = align_df.join(
                age_df[['age_at_research_stage', 'sex']].droplevel('array_index'))\
                .rename(columns={'age_at_research_stage': 'age'})
            self.fields += ['age', 'sex']
            return

        date_cols = np.array(['collection_timestamp', 'collection_date', 'sequencing_date'])
        date = date_cols[np.isin(date_cols, align_df.columns)][0]  # prefer first match

        age_df['birth_date'] = pd.to_datetime(
            age_df['year_of_birth'].astype(str) + '-' + age_df['month_of_birth'].astype(str))

        self.dfs['age_sex'] = align_df[[date]].join(age_df[['sex', 'birth_date']])\
            .assign(age=lambda x: ((x[date].dt.date - x['birth_date'].dt.date).dt.days / 365.25).round(1))\
            .drop(columns=['birth_date'])
        self.fields += ['age', 'sex']

    def __load_dataframes__(self) -> None:
        """
        Load all tables in the dataset dictionary.
        """
        self.dfs = {}
        self.fields = set()
        for relative_location in self.dict['relative_location'].dropna().unique():
            if any([pattern in relative_location for pattern in self.skip_dfs]):
                print(f'Skipping {relative_location}')
                continue
            self.dfs[relative_location.split('.')[0]] = self.__load_one_dataframe__(relative_location)
            self.fields |= set(self.dfs[relative_location.split('.')[0]].columns.tolist())
        self.fields = list(self.fields)

    def __load_one_dataframe__(self, relative_location: str) -> pd.DataFrame:
        """
        Load one dataframe.

        Args:
            relative_location (str): the location of the dataframe

        Returns:
            pd.DataFrame: the loaded dataframe
        """
        df_path = os.path.join(
            self.base_path,
            self.dataset,
            self.cohort,
            relative_location)
        try:
            data =  pd.read_parquet(df_path)
        except Exception as err:
            if self.errors == 'raise':
                raise err
            if self.errors == 'warn':
                warnings.warn(f'Error loading {df_path}:\n{err}')
            return pd.DataFrame()

        # set the order of columns according to the dictionary
        dict_columns = self.dict.index.intersection(data.columns)
        other_columns = data.columns.difference(self.dict.index)
        assert (len(dict_columns) + len(other_columns)) == len(data.columns), "something isn't right"
        data = data[dict_columns.tolist() + other_columns.tolist()]

        before = len(data)
        if self.unique_index:
            data = data.loc[~data.index.duplicated()]
        if self.valid_dates:
            data = data.loc[data.select_dtypes(include=['datetime64[ns]']).notnull().any(axis=1)]
        if self.valid_stage:
            data = data.loc[data.index.get_level_values('research_stage').notnull()]
        after = len(data)
        if before > after:
            print(f'Filtered {before - after} rows')

        return data

    def __load_dictionary__(self) -> None:
        """
        Load dataset dictionary.
        """
        self.dict = pd.read_csv(self.__get_file_path__(self.dataset, 'csv'))\
            .set_index('tabular_field_name')
        self.fields = self.dict.index.tolist()

    def __get_file_path__(self, dataset: str, extension: str) -> str:
        """
        Get the file path for a dataset and an extension.

        Args:
            dataset (str): the name of the dataset
            extension (str): the extension of the file

        Returns:
            str: the path to the file
        """
        path = os.path.join(self.base_path, dataset, self.cohort, '*.' + extension)
        if path.startswith('s3://'):
           return path
        return glob(path)[0]

In [5]:
dl = DataLoader('human_genetics', base_path='s3://phenodatasets', age_sex_dataset='population',
                errors='warn', skip_dfs=['relatives'])

Skipping relatives/plink_ibs.parquet
Skipping relatives/king_kinship.parquet


/tmp/ipykernel_73434/2248394877.py:250: UserWarning: Error loading s3://phenodatasets/human_genetics/10k/pca/eigenvec.var:
Could not open Parquet input source 'phenodatasets/human_genetics/10k/pca/eigenvec.var': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
  warnings.warn(f'Error loading {df_path}:\n{err}')


In [11]:
dl.dfs['age_sex']

collection_date  sex   age
participant_id cohort array_index research_stage                           
1000942861     10k    0           00_00_visit         2022-05-16    1  54.8
                                  01_00_call          2022-05-16    1  54.8
1001201093     10k    0           00_00_visit         2021-11-24    0  42.3
                                  01_00_call          2021-11-24    0  42.3
1002033709     10k    0           00_00_visit         2022-11-30    0  43.2
...                                                          ...  ...   ...
9999226141     10k    0           00_00_visit         2021-07-27    1  43.1
                                  03_00_call          2021-07-27    1  43.1
                                  01_00_call          2021-07-27    1  43.1
9999409119     10k    0           00_00_visit         2020-09-23    0  54.1
                                  01_00_call          2020-09-23    0  54.1

[19706 rows x 3 columns]

In [12]:
dl.dfs['main'].loc[1000942861]

,,collection_date,version,genecov_qc_bases,genecov_qc_bases_dedup,gencove_qc_bases_dedup_mapped,genecov_qc_effective_coverage,genecov_qc_fraction_contamination,genecov_qc_snps,genecov_qc_format_passed,genecov_qc_r1_eq_r2_passed,...,postqc_bim_chr_19,postqc_bed_chr_20,postqc_fam_chr_20,postqc_bim_chr_20,postqc_bed_chr_21,postqc_fam_chr_21,postqc_bim_chr_21,postqc_bed_chr_22,postqc_fam_chr_22,postqc_bim_chr_22
cohort,array_index,,,,,,,,,,,,,,,,,,,,,
10k,0,2022-05-16,v02,2.048679e+09,1.806727e+09,1.691509e+09,0.385886,0.005,27119329.0,True,True,...,variants_postqc/chr_19.bim,variants_postqc/chr_20.bed,variants_postqc/chr_20.fam,variants_postqc/chr_20.bim,variants_postqc/chr_21.bed,variants_postqc/chr_21.fam,variants_postqc/chr_21.bim,variants_postqc/chr_22.bed,variants_postqc/chr_22.fam,variants_postqc/chr_22.bim


In [23]:
dl.load_sample_data('postqc_fam_chr_22', 6104973108)

KeyboardInterrupt: 

Use the dataset name to load the dataset. It may contain multiple tables. Age / sex will be added to the data by default. The default `base_path` is set to work on the research platform.

In [ ]:
dl = DataLoader('Fundus', base_path='examples/', errors='warn')
dl

DataLoader for Fundus with
78 fields
2 tables: ['fundus', 'age_sex']

The data dictionary of the dataset displays the description of each field.

In [ ]:
dl.dict.head(3)

,field_string,description_string,parent_dataframe,relative_location,value_type,units,sampling_rate,item_type,array,cohorts,data_type,debut,pandas_dtype
tabular_field_name,,,,,,,,,,,,,
fundus_image_left,Fundus image (left),Fundus image (left),NaN,fundus.parquet,Text,None,NaN,Bulk,Single,10K,image,2021-02-17,string
fundus_image_right,Fundus image (right),Fundus image (right),NaN,fundus.parquet,Text,None,NaN,Bulk,Single,10K,image,2021-02-17,string
collection_date,Collection date (YYYY-MM-DD),Collection date (YYYY-MM-DD),NaN,fundus.parquet,Date,Time,NaN,Data,Single,10K,tabular,2021-02-17,datetime64[ns]


In [ ]:
dl.dfs['fundus'].head(3)

,,,,fundus_image_left,fundus_image_right,collection_date,artery_average_width_left,artery_average_width_right,artery_distance_tortuosity_left,artery_distance_tortuosity_right,artery_fractal_dimension_left,artery_fractal_dimension_right,artery_squared_curvature_tortuosity_left,...,vein_fractal_dimension_left,vein_fractal_dimension_right,vein_squared_curvature_tortuosity_left,vein_squared_curvature_tortuosity_right,vein_tortuosity_density_left,vein_tortuosity_density_right,vein_vessel_density_left,vein_vessel_density_right,vessel_density_left,vessel_density_right
participant_id,cohort,research_stage,array_index,,,,,,,,,,,,,,,,,,,,,
0,10k,00_00_visit,0,/path/to/file,/path/to/file,2022-11-16,18430.284751,19038.547771,3.668175,3.271147,1.355673,1.343602,40.648267,...,1.410553,1.403108,14.208195,6.098432,0.700187,0.698546,0.046645,0.045864,0.080377,0.078671
1,10k,00_00_visit,0,/path/to/file,/path/to/file,2022-06-30,17315.398780,19099.489575,2.095461,1.634782,1.368933,1.363413,24.253169,...,1.387527,1.332864,8.999069,8.702682,0.740806,0.708911,0.037896,0.046853,0.074197,0.064578
2,10k,00_00_visit,0,/path/to/file,/path/to/file,2021-10-05,15375.866993,19855.576862,2.776472,2.747015,1.360404,1.362699,9.742353,...,1.411881,1.408791,13.119227,9.936669,0.627281,0.675100,0.053022,0.048063,0.079515,0.082102


All availbale fields (columns) in all tables can be listed.

In [ ]:
dl.fields[:5]

['artery_squared_curvature_tortuosity_right',
 'fundus_image_binary_vein_segmentation_right',
 'timezone',
 'fundus_image_quality_good_score_left',
 'fundus_image_quality_prediction_left']

Access any of the fields (e.g., `vein_average_width_right`, `age`) or indices (e.g., `research_stage`) from any of the tables via the data loader API.

In [ ]:
dl[['research_stage', 'vein_average_width_right', 'age', 'sex']]

,,,,vein_average_width_right,age,sex,research_stage
participant_id,cohort,research_stage,array_index,,,,
0,10k,00_00_visit,0,18436.428634,43.5,0,00_00_visit
1,10k,00_00_visit,0,18888.160314,53.7,1,00_00_visit
2,10k,00_00_visit,0,19013.865043,26.2,0,00_00_visit
3,10k,00_00_visit,0,18809.012493,44.6,1,00_00_visit
4,10k,00_00_visit,0,19428.986690,50.3,0,00_00_visit


Access time series or bulk data that is stored separately for each sample via the data loader API. In the following example, the data loader retrieves the relative path of each sample's bulk file from the main table (where it is stored in the field `fundus_image_left`), converts it to an absolute path, and loads the file. This is repeated for 3 samples and returned as a list. In the case of parquet DataFrames, there is no need to define the `load_func` and the results are concatenated by deafult.

In [ ]:
from PIL import Image

In [ ]:
#| hide

dl.dfs['fundus']['fundus_image_left'] = [f'M0/images/fundus_{i}.png' for i in range(5)]

In [ ]:
dl.load_sample_data('fundus_image_left', [0, 1, 2], load_func=Image.open, concat=False)

[<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=25x25>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=25x25>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=25x25>]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()